# Batch run models

In [ ]:
%load_ext lab_black
import os, multiprocessing, time
import papermill as pm
from meta import model_cfg, make_batch_cfg, parse_batch_results

## Make configs

In [ ]:
batch_name = "O2P_m270_3other_injection_timing"
batch_output_dir = "batch_eval/{}/".format(batch_name)

# Gen OG random seed: [int(random.random() * 1e5) for x in range(10)]
# Below using the same seed as m270, but due to dictionary change, it is qualitatively not the same anymore

param_grid = {
    "p_noise": [0.0, 1.0, 3.0],
    "hidden_units": [50, 100, 250],
    "learning_rate": [0.05, 0.01, 0.02],
    "output_ticks": [1, 6, 11],
    "rng_seed":
        [53797, 77501, 86017, 61301, 96533, 88467, 85362, 66721, 49596, 71461]
}

static_hpar = {
    "sample_name": "jay",
    "cleanup_units": 20,
    "use_semantic": False,
    "input_dim": 119,
    "output_dim": 250,
    "pretrain_attractor": False,
    "rnn_activation": "sigmoid",
    "regularizer_const": None,
    "w_initializer": "glorot_uniform",
    "tau": 0.2,
    "max_unit_time": 4.0,
    "optimizer": "adam",
    "n_mil_sample": 1.0,
    "batch_size": 128,
    "save_freq": 10,
    "bq_dataset": batch_name,
}

batch_cfgs = make_batch_cfg(
    batch_name, static_hpar, param_grid, "OSP_master.ipynb"
)

## Parallel run

In [ ]:
# Run
def run_batch(cfg):
    """
    Using papermill to run parameterized notebook
    """

    if not os.path.isdir(cfg['model_folder']):
        print("Running model {}".format(cfg['sn']))
        os.makedirs(cfg['model_folder'], exist_ok=True)
        pm.execute_notebook(
            cfg['in_notebook'], cfg['out_notebook'], parameters=cfg['params']
        )


# Run in parallel pool
with multiprocessing.Pool(4) as pool:
    pool.map(run_batch, batch_cfgs)

Compile and save results

In [ ]:
cfgs, df = parse_batch_results(batch_cfgs)
df.to_csv(batch_output_dir + 'bcdf.csv')
cfgs.to_csv(batch_output_dir + 'cfgs.csv')

Shutdown compute engine

In [ ]:
time.sleep(30)
print('shuting down')
!sudo poweroff